In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np



In [3]:
# Function to prepare custom text data
def prepare_data(text_data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text_data])
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in text_data.split('.'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)

    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = tf.keras.utils.to_categorical(y, num_classes=total_words)

    return X, y, tokenizer, max_sequence_len, total_words

In [14]:
# User input for custom text
custom_text = input("Enter the text data for training (e.g., 'Naruto is a ninja from the Hidden Leaf Village.'): ")
X, y, tokenizer, max_sequence_len, total_words = prepare_data(custom_text)

In [16]:
# Build the LSTM Model for Text Generation
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len - 1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
# 3. Train the Model
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 5.6836
Epoch 2/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 5.2361
Epoch 3/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 5.1698
Epoch 4/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 5.1476
Epoch 5/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 5.1158
Epoch 6/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 5.0493
Epoch 7/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 4.9990
Epoch 8/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 4.8674
Epoch 9/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 4.7991
Epoch 10/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 4.6045
Epoch 11/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 4.4291
Epoch 12/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 4.2154
Epoch 13/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 4.0354
Epoch 14/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 3.8383
Epoch 15/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - lo

In [18]:
# 4. Generate Text using the Trained Model
def generate_text(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [15]:
# 5. Test the Text Generation
seed_text_input = input("Enter a seed text for generation (e.g., 'Naruto'): ")
print(generate_text(seed_text_input, 10))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Iruka playing pranks mizuki the a instead attending already failed last


In [ ]:
Naruto Uzumaki is a young boy from the Hidden Leaf Village and the biggest problem child at the ninja academy. He causes mischief every day and is constantly being scolded by his teacher, Iruka Umino. He has fallen so far behind in his studies that his graduation is uncertain, but Naruto actually dreams of becoming the leader of the village, known as the Hokage, and vows to one day become the No. 1 ninja.
However, all the adults in the village treat Naruto with disdain due to a secret surrounding his birth. 12 years ago, when Naruto was just a baby, the Nine-Tailed Fox attacked the village, and the only way to stop it was to seal it inside of him. This fact is first revealed to him by Mizuki, an instructor at the academy who uses Naruto to steal the Scroll of Sealing from the current Hokage. Once the deed is done, he appears before the young ninja and discloses the shocking truth behind his birth. Naruto manages to defeat Mizuki with Iruka’s help, who despite knowing about Naruto’s past, has always believed in him. After learning Iruka’s true feelings, Naruto gains a renewed determination to pursue his dream.
After successfully graduating from the academy alongside fellow students Sasuke Uchiha and Sakura Haruno, Naruto finally takes the first steps on his journey as a real ninja!


